<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.1: Introduction to FIRRTL

**Prev: [Generators: Types](3.6_types.ipynb)**<br>
**Next: [FIRRTL AST Traversal](4.2_firrtl_ast_traversal.ipynb)**

## Motivation
Scala를 배우고 Chisel을 작성했으며 사용자의 90%는 Chisel 애호가가 되기에 충분할 것입니다.

그러나 일부 사용 사례는 생성기보다는 치즐 디자인의 프로그래밍 방식 변환으로 더 잘 표현됩니다.

예를 들어, 디자인의 레지스터 수를 세고 싶다고 가정합니다. 이것은 생성기로 수행하기 어려울 수 있으므로 대신 FIRRTL 패스를 작성하여 수행할 수 있습니다.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp-kr/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import firrtl._

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import firrtl._

## What is FIRRTL?
아시다시피, Chisel 디자인을 실행할 때 모든 Scala 매개변수가 해결된 상태에서 생성기의 인스턴스를 구성하기 위해 정교하게(주변의 Scala 코드를 실행합니다.)

Verilog를 직접 내보내는 대신 Chisel은 정교한(매개변수 해결) RTL 인스턴스를 나타내는 FIRRTL이라는 중간 표현을 내보냅니다. 직렬화(파일에 쓰기 위해 문자열로 변환)될 수 있으며 이 직렬화된 구문은 사람이 읽을 수 있습니다. 그러나 내부적으로는 긴 문자열로 표시되지 않습니다. 대신 AST(abstract-syntax-tree)라고 하는 노드 트리로 구성된 데이터 구조입니다.

한 번 보자! 우리는 간단한 끌 디자인을 가져와 정교하게 만들고 그것이 생성하는 FIRRTL을 조사할 것입니다!

먼저 입력 신호를 2사이클 지연시키는 치즐 모듈을 정의합니다.

In [3]:
class DelayBy2(width: Int) extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  val r0 = RegNext(io.in)
  val r1 = RegNext(r0)
  io.out := r1
}

defined class DelayBy2

다음으로, 생성한 FIRRTL을 정교화하고 직렬화하고 인쇄해 보겠습니다.

In [4]:
println(chisel3.Driver.emit(() => new DelayBy2(4)))

Elaborating design...
Done elaborating.
circuit DelayBy2 :
  module DelayBy2 :
    input clock : Clock
    input reset : UInt<1>
    output io : { flip in : UInt<4>, out : UInt<4>}

    reg r0 : UInt, clock with :
      reset => (UInt<1>("h0"), r0) @[cmd2.sc 6:19]
    r0 <= io.in @[cmd2.sc 6:19]
    reg r1 : UInt, clock with :
      reset => (UInt<1>("h0"), r1) @[cmd2.sc 7:19]
    r1 <= r0 @[cmd2.sc 7:19]
    io.out <= r1 @[cmd2.sc 8:10]



보시다시피 직렬화된 FIRRTL은 모든 생성기 매개변수가 해결된 치즐 디자인과 매우 유사합니다.

## The FIRRTL AST

앞에서 언급했듯이 FIRRTL 표현은 문자열로 직렬화될 수 있지만 내부적으로는 AST(추상 구문 트리)라는 데이터 구조입니다. 이 데이터 구조는 한 노드가 자식 노드를 포함할 수 있는 노드 트리입니다. 이 데이터 구조에는 주기가 없습니다.

내부 데이터 구조가 어떻게 생겼는지 살펴보겠습니다.

In [5]:
val firrtlSerialization = chisel3.Driver.emit(() => new DelayBy2(4))
val firrtlAST = firrtl.Parser.parse(firrtlSerialization.split("\n").toIterator, Parser.GenInfo("file.fir"))

println(firrtlAST)

Elaborating design...
Done elaborating.
Circuit( @[file.fir 1:0],ArrayBuffer(Module( @[file.fir 2:2],DelayBy2,ArrayBuffer(Port( @[file.fir 3:4],clock,Input,ClockType), Port( @[file.fir 4:4],reset,Input,UIntType(IntWidth(1))), Port( @[file.fir 5:4],io,Output,BundleType(ArrayBuffer(Field(in,Flip,UIntType(IntWidth(4))), Field(out,Default,UIntType(IntWidth(4))))))),Block(ArrayBuffer(DefRegister( @[file.fir 7:4],r0,UIntType(UnknownWidth),Reference(clock,UnknownType,UnknownKind,UnknownFlow),UIntLiteral(0,IntWidth(1)),Reference(r0,UnknownType,UnknownKind,UnknownFlow)), Connect( @[file.fir 9:4],Reference(r0,UnknownType,UnknownKind,UnknownFlow),SubField(Reference(io,UnknownType,UnknownKind,UnknownFlow),in,UnknownType,UnknownFlow)), DefRegister( @[file.fir 10:4],r1,UIntType(UnknownWidth),Reference(clock,UnknownType,UnknownKind,UnknownFlow),UIntLiteral(0,IntWidth(1)),Reference(r1,UnknownType,UnknownKind,UnknownFlow)), Connect( @[file.fir 12:4],Reference(r1,UnknownType,UnknownKind,UnknownFlow),Ref

firrtlSerialization: String = """circuit DelayBy2 :
  module DelayBy2 :
    input clock : Clock
    input reset : UInt<1>
    output io : { flip in : UInt<4>, out : UInt<4>}

    reg r0 : UInt, clock with :
      reset => (UInt<1>("h0"), r0) @[cmd2.sc 6:19]
    r0 <= io.in @[cmd2.sc 6:19]
    reg r1 : UInt, clock with :
      reset => (UInt<1>("h0"), r1) @[cmd2.sc 7:19]
    r1 <= r0 @[cmd2.sc 7:19]
    io.out <= r1 @[cmd2.sc 8:10]
"""
firrtlAST: ir.Circuit = Circuit(
  FileInfo("file.fir 1:0"),
  ArrayBuffer(
    Module(
      FileInfo("file.fir 2:2"),
      "DelayBy2",
      ArrayBuffer(
        Port(FileInfo("file.fir 3:4"), "clock", Input, ClockType),
        Port(FileInfo("file.fir 4:4"), "reset", Input, UIntType(IntWidth(1))),
        Port(
          FileInfo("file.fir 5:4"),
          "io",
          Output,
          BundleType(
            ArrayBuffer(
              Field("in", Flip, UIntType(IntWidth(4))),
              Field("out", Default, UIntType(IntWidth(4)))
            

분명히 데이터 구조의 직렬화는 그다지 아름답지 않지만 내부적으로 RTL 디자인을 나타내는 클래스 등을 볼 수 있습니다. 이해를 돕기 위해 조금 더 예쁘게 만들어 보겠습니다.

In [6]:
println(stringifyAST(firrtlAST))

Circuit(
| @[file.fir1:0],
| ArrayBuffer(
| | Module(
| | | @[file.fir2:2],
| | | DelayBy2,
| | | ArrayBuffer(
| | | | Port(
| | | | | @[file.fir3:4],
| | | | | clock,
| | | | | Input,
| | | | | ClockType
| | | | ),
| | | | Port(
| | | | | @[file.fir4:4],
| | | | | reset,
| | | | | Input,
| | | | | UIntType(
| | | | | | IntWidth(
| | | | | | | 1
| | | | | | )
| | | | | )
| | | | ),
| | | | Port(
| | | | | @[file.fir5:4],
| | | | | io,
| | | | | Output,
| | | | | BundleType(
| | | | | | ArrayBuffer(
| | | | | | | Field(
| | | | | | | | in,
| | | | | | | | Flip,
| | | | | | | | UIntType(
| | | | | | | | | IntWidth(
| | | | | | | | | | 4
| | | | | | | | | )
| | | | | | | | )
| | | | | | | ),
| | | | | | | Field(
| | | | | | | | out,
| | | | | | | | Default,
| | | | | | | | UIntType(
| | | | | | | | | IntWidth(
| | | | | | | | | | 4
| | | | | | | | | )
| | | | | | | | )
| | | | | | | )
| | | | | | )
| | | | | )
| | | | )
| | | ),
| | | Block(
| | | | ArrayBuffer(
| | | | | DefRegister(
| |

이것은 FIRRTL AST를 보유하는 내부 데이터 구조입니다. 루트 노드가 **Circuit**인 트리 구조로 **@[file.fir@2.0]**, **ArrayBuffer**, **cmd5WrapperHelperDelayBy2**의 3개의 자식이 있습니다. 다음은 직렬화된 `Circuit`의 실제 Scala 클래스의 정의입니다. <a name="circuit"></a><img src="images/circuit.png" alt="Circuit case class" />



보시다시피, `info: Info`, `Modules: Seq[DefModule]`, `main: String`의 세 가지 자식 노드가 있습니다. 모든 FIRRTL AST 노드가 수행해야 하는 'FirrtlNode'를 확장합니다. 지금은 `def mapXXXX` 기능을 무시하십시오.

많은 FIRRTL 노드에는 파서가 줄 번호 및 열 번호와 같은 파일 정보를 삽입하거나 'NoInfo' 토큰을 삽입할 수 있는 `info: Info` 필드가 포함되어 있습니다. 이 예에서 **@[file.fir@2.0]**은 FIRRTL 파일, 2행, 0열을 나타냅니다.

다음 섹션에서는 이러한 모든 FIRRTL 노드에 대해 자세히 설명합니다.

# FIRRTL Node Descriptions

이 섹션에서는 [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/ucb-bar/firrtl/blob/master/src/main/scala/firrtl)에 있는 일반적인 FirrtlNode에 대해 설명합니다. /ir/IR.scala).

여기에 언급되지 않은 구성 요소에 대한 자세한 내용은 [The FIRRTL 사양](https://github.com/ucb-bar/firrtl/blob/master/spec/spec.pdf)을 참조하십시오.


## Circuit
회로는 모든 Firrtl 데이터 구조의 루트 노드입니다. 하나의 회로만 있으며 해당 회로에는 모듈 정의 목록과 최상위 모듈 이름이 포함되어 있습니다.

#### FirrtlNode Declaration
```scala 
Circuit(info: Info, modules: Seq[DefModule], main: String)
```

#### Concrete Syntax
```
circuit Adder:
  ... //List of modules
```
#### In-memory Representation
```scala
Circuit(NoInfo, Seq(...), "Adder")
```

## Module

모듈은 Firrtl 내의 모듈화 단위이며 직접 중첩되지 않습니다(모듈의 인스턴스 선언에는 고유한 구체적인 구문과 AST 표현이 있습니다). 각 모듈에는 이름, 포트 목록 및 구현을 포함하는 본문이 있습니다.

#### FirrtlNode declaration
```scala
Module(info: Info, name: String, ports: Seq[Port], body: Stmt) extends DefModule
```

#### Concrete Syntax
```
module Adder:
  ... // list of ports
  ... // statements
```
#### In-memory representation
```scala
Module(NoInfo, "Adder", Seq(...), )
```

## Port
포트는 모듈 io의 일부를 정의하며 이름, 방향(입력 또는 출력) 및 유형이 있습니다.

#### FirrtlNode Declaration
```scala
class Port(info: Info, name: String, direction: Direction, tpe: Type)
```
#### Concrete Syntax
```
input x: UInt
```

#### In-memory representation
```scala
Port(NoInfo, "x", INPUT, UIntType(UnknownWidth))
```

## Statement
문은 모듈 내의 구성 요소와 상호 작용 방식을 설명하는 데 사용됩니다. 다음은 일반적으로 사용되는 몇 가지 구문입니다.

### Block of Statements
명령문 그룹입니다. 일반적으로 모듈 선언에서 본문 필드로 사용됩니다.

### Wire Declaration
이름과 유형을 포함하는 와이어 선언. 소스(*from* 연결)와 싱크(*to" 연결)가 될 수 있습니다.
#### FirrtlNode declaration
```scala
DefWire(info: Info, name: String, tpe: Type)
```
#### Concrete syntax
```
wire w: UInt
```
#### In-memory Representation
```scala
DefWire(NoInfo, "w", UIntType(UnknownWidth))
```

### Register Declaration
이름, 유형, 클록 신호, 재설정 신호 및 재설정 값을 포함하는 레지스터 선언.
#### FirrtlNode declaration
```scala
DefRegister(info: Info, name: String, tpe: Type, clock: Expression, reset: Expression, init: Expression)
```

### Connection
소스에서 싱크로의 방향 연결을 나타냅니다. Chisel에 설명된 대로 last-connect-semantics를 준수합니다.

#### FirrtlNode declaration
```scala
Connect(info: Info, loc: Expression, expr: Expression)
```

### Other Statements
'DefMemory', 'DefNode', 'IsInvalid', 'Conditionally' 등과 같은 다른 문 유형은 여기에서 생략됩니다. 자세한 내용은 [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/freechipsproject/firrtl/blob/master/src/main/scala/firrtl/ir/IR.scala)를 참조하십시오.

## Expression
식은 선언된 구성 요소 또는 논리 및 산술 연산에 대한 참조를 나타냅니다. 다음은 일반적으로 사용되는 몇 가지 표현입니다.

### Reference
와이어, 레지스터 또는 포트와 같은 선언된 구성 요소에 대한 참조입니다. 이름과 유형 필드가 있습니다. 여기에는 실제 선언에 대한 포인터가 포함되어 있지 않고 대신 문자열로 이름만 포함되어 있습니다.

#### FirrtlNode declaration
```scala
Reference(name: String, tpe: Type)
```

### DoPrim
'Add', 'Sub', 'And', 'Or' 또는 하위 단어 선택('Bits')과 같은 익명의 기본 작업입니다. 작업 유형은 'op: PrimOp' 필드로 표시됩니다. 필요한 인수와 상수의 수는 `op`에 의해 결정됩니다.

#### FirrtlNode declaration
```scala
DoPrim(op: PrimOp, args: Seq[Expression], consts: Seq[BigInt], tpe: Type)
```

### Other Expressions
Other expressions including `SubField`, `SubIndex`, `SubAccess`, `Mux`, `ValidIf` etc. are described in more detail in [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/ucb-bar/firrtl/blob/master/src/main/scala/firrtl/ir/IR.scala) and [The FIRRTL Specification](https://github.com/ucb-bar/firrtl/blob/master/spec/spec.pdf).

# Back to our example

예제에서 FIRRTL AST를 다시 살펴보겠습니다. 바라건대, 디자인의 구조가 더 의미가 있습니다!

In [7]:
println(stringifyAST(firrtlAST))

Circuit(
| @[file.fir1:0],
| ArrayBuffer(
| | Module(
| | | @[file.fir2:2],
| | | DelayBy2,
| | | ArrayBuffer(
| | | | Port(
| | | | | @[file.fir3:4],
| | | | | clock,
| | | | | Input,
| | | | | ClockType
| | | | ),
| | | | Port(
| | | | | @[file.fir4:4],
| | | | | reset,
| | | | | Input,
| | | | | UIntType(
| | | | | | IntWidth(
| | | | | | | 1
| | | | | | )
| | | | | )
| | | | ),
| | | | Port(
| | | | | @[file.fir5:4],
| | | | | io,
| | | | | Output,
| | | | | BundleType(
| | | | | | ArrayBuffer(
| | | | | | | Field(
| | | | | | | | in,
| | | | | | | | Flip,
| | | | | | | | UIntType(
| | | | | | | | | IntWidth(
| | | | | | | | | | 4
| | | | | | | | | )
| | | | | | | | )
| | | | | | | ),
| | | | | | | Field(
| | | | | | | | out,
| | | | | | | | Default,
| | | | | | | | UIntType(
| | | | | | | | | IntWidth(
| | | | | | | | | | 4
| | | | | | | | | )
| | | | | | | | )
| | | | | | | )
| | | | | | )
| | | | | )
| | | | )
| | | ),
| | | Block(
| | | | ArrayBuffer(
| | | | | DefRegister(
| |

그것이 이 섹션에 대한 것입니다! 다음 섹션에서는 FIRRTL 변환이 이 AST를 어떻게 이동하고 수정하는지 살펴보겠습니다.